<a href="https://colab.research.google.com/github/hertie-data-science-lab/tutorial-new-group-2-1/blob/main/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Transfer Learning for Flood Mapping Using Sentinel-1 Radar Imagery


# GRAD-E1394 Deep Learning - Assignment 3

Authors:


*   Aditi Joshi
*   Elena Murray
*   Leticia Figueiredo Collado
*   Sattiki Ganguly
*   Xiaohan Wu







# Memo - add more context

Flooding is one of the most damaging climate-related hazards today. It destroy infrastructure, reduce agricultural output, and threaten lives. Governments increasingly rely on satellite imagery to monitor floods, especially in regions lacking ground sensors. Sentinel-1, a freely available radar satellite, is particularly valuable because it can capture images regardless of cloud cover, exactly when floods occur.

However, interpreting satellite radar data requires technical expertise and manual processing. Deep learning models can help analysts produce rapid, large-scale flood maps, supporting emergy response, disaster insurance, infrastructure planning, and climate adaptation strategies.

This tutorial introduces transfer learning, a method that reuses knowledge learned by large pretrained CNNs and adapts them to a new task with limited labeled data. This approach is widely used in governments, including FEMA (USA), the European Copernicus Program, and India's National Remote Sensing Center.

This tutorial provides a practical and reproducible workflow that supports flood-risk monitoring, climate-resilience planning, and rapid diasaster assessment. It equips analysts with the foundational skills needed to support data-driven policy in environmental management and emergency decision-making.

## 1. Overview

to be added

## 2. Background & Prerequisites

### Conceptual background

* **Convolutional Neural Networks (CNNs):**  
  Neural networks that apply learnable filters across images to detect patterns like edges, textures, and shapes.

* **Transfer Learning:**  
  Instead of training a CNN from scratch, we:
  1. Start from a model pretrained on a large dataset (e.g. ImageNet with millions of natural images).
  2. Replace and fine-tune only the last layers for our task.
  3. Benefit from previously learned low-level features, saving data and compute.

* **Sentinel-1 Radar (SAR):**  
  A satellite that sends microwave pulses and measures backscatter. Flooded areas often appear darker because smooth water surfaces reflect energy away from the satellite.


## 3. Software Requirements

This notebook is designed for **Google Colab**, but it also works in a local Jupyter environment if you have access to the data.

We use:

* `torch`, `torchvision` – deep learning
* `numpy`, `pandas` – data handling
* `rasterio` – reading GeoTIFF satellite images
* `matplotlib` – plotting
* `sklearn` – splitting data, metrics

Run the following cell once to install dependencies.

In [12]:
!pip install rasterio
!pip install torch torchvision torchaudio
!pip install scikit-learn

In [1]:
!pip install -q rasterio geopandas matplotlib seaborn scikit-learn torch torchvision

# Import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import rasterio
from rasterio.plot import show
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import json
from collections import Counter

## 4. Data Description – Sen1Floods11 (Small Subset)

We work with a **small sample** of the [Sen1Floods11](http://openaccess.thecvf.com/content_CVPRW_2020/html/w11/Bonafilia_Sen1Floods11_A_Georeferenced_Dataset_to_Train_and_Test_Deep_Learning_CVPRW_2020_paper.html) dataset.

Each *event* consists of:

* **S1**: Sentinel-1 radar image tile (512×512, bands: VV & VH).
* **QC**: Hand-labeled ground truth mask (512×512), values:
  * `1` = water (flooded)
  * `0` = non-water
  * `-1` = no data / invalid


In [10]:
#rm -rf /content/sen1floods11_subset

In [11]:
from pathlib import Path

DATA_ROOT = Path("/content/sen1floods11_subset")
(DATA_ROOT).mkdir(parents=True, exist_ok=True)

(DATA_ROOT / "catalog").mkdir(parents=True, exist_ok=True)
(DATA_ROOT / "S1").mkdir(exist_ok=True)
(DATA_ROOT / "QC").mkdir(exist_ok=True)
(DATA_ROOT / "S2").mkdir(exist_ok=True)

DATA_ROOT

PosixPath('/content/sen1floods11_subset')

In [12]:
ls -R /content/sen1floods11_subset

/content/sen1floods11_subset:
catalog/  QC/  S1/  S2/

/content/sen1floods11_subset/catalog:

/content/sen1floods11_subset/QC:

/content/sen1floods11_subset/S1:

/content/sen1floods11_subset/S2:


In [13]:
!gsutil ls gs://sen1floods11/v1.1/

gs://sen1floods11/v1.1/Sen1Floods11_Metadata.geojson
gs://sen1floods11/v1.1/catalog.zip
gs://sen1floods11/v1.1/catalog/
gs://sen1floods11/v1.1/checkpoints/
gs://sen1floods11/v1.1/data/
gs://sen1floods11/v1.1/splits/


In [14]:
!gsutil cp gs://sen1floods11/v1.1/Sen1Floods11_Metadata.geojson /content/sen1floods11_subset/

Copying gs://sen1floods11/v1.1/Sen1Floods11_Metadata.geojson...
/ [1 files][ 13.8 KiB/ 13.8 KiB]                                                
Operation completed over 1 objects/13.8 KiB.                                     


In [16]:
!gsutil -m cp -r gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label \
    /content/sen1floods11_subset/catalog/

Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_195474_label/Bolivia_195474_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_103757_label/Bolivia_103757_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_314919_label/Bolivia_314919_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_242570_label/Bolivia_242570_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_312675_label/Bolivia_312675_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_290290_label/Bolivia_290290_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_23014_label/Bolivia_23014_label.json...
Copying gs://sen1floods11/v1.1/catalog/sen1floods11_hand_labeled_label/Bolivia_294583_label/Bolivia_294583_label.json...
Copying gs://sen1floods11/v1.1/cat

In [17]:
ls /content/sen1floods11_subset/catalog/sen1floods11_hand_labeled_label | head

Bolivia_103757_label/
Bolivia_129334_label/
Bolivia_195474_label/
Bolivia_23014_label/
Bolivia_233925_label/
Bolivia_242570_label/
Bolivia_290290_label/
Bolivia_294583_label/
Bolivia_312675_label/
Bolivia_314919_label/


In [18]:
import json, glob, random

json_files = glob.glob("/content/sen1floods11_subset/catalog/**/*.json", recursive=True)
subset_json = random.sample(json_files, 10)